In [ ]:
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from statsmodels.tools.sm_exceptions import ValueWarning, HessianInversionWarning, ConvergenceWarning
import warnings

#in practice do not supress these warnings, they carry important information about the status of your model
warnings.filterwarnings('ignore', category=ValueWarning)
warnings.filterwarnings('ignore', category=HessianInversionWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)

In [ ]:
tickerSymbol = 'GC=F'
data = yf.Ticker(tickerSymbol)

prices = data.history(start='2020-01-01', end='2024-05-01').Close
returns = prices.pct_change().dropna()

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(returns)
plt.ylabel('Return', fontsize=20)

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(prices)
plt.ylabel('Price', fontsize=20)

In [ ]:
plot_acf(returns)
plt.show()

In [ ]:
plot_pacf(returns)
plt.show()

In [ ]:
def run_simulation(returns, prices, amount, order, thresh, verbose=True, plot=True):
    returns.index = pd.to_datetime(returns.index)

    #Rolling window of lags for ARIMA
    window_size = 5

    curr_holding = False
    events_list = []
    init_amount = amount

    predictions = returns.copy()

    for i in tqdm(range(window_size, len(returns)), desc='Processing predictions'):
        tmrw_date = returns.index[i]
        today_date = returns.index[i-1]
        curr_data = returns.iloc[i-window_size:i]
        model = ARIMA(curr_data, order=order)
        results = model.fit()
        prediction = results.forecast(1)
        predictions.iloc[i] = prediction.iloc[0]

        if (not curr_holding) and (prediction.iloc[0] > thresh):
            buy_price = prices.iloc[i-1]
            curr_holding = True
            events_list.append(('b', today_date))
        elif (prediction.iloc[0] < 0 or i == len(returns)-1) and curr_holding:
            sell_price = prices.iloc[i-1]
            # if sell_price > buy_price:
            ret = (sell_price - buy_price)/buy_price
            amount += ret
            curr_holding = False
            events_list.append(('s', today_date, ret))


    if plot:
        plt.figure(figsize=(12,6))
        plt.plot(prices[14:])

        y_lims = (int(prices.min()*.95), int(prices.max()*1.05))
        shaded_y_lims = int(prices.min()*.5), int(prices.max()*1.5)

        for idx, event in enumerate(events_list):
            plt.axvline(event[1], color='k', linestyle='--', alpha=0.4)
            if event[0] == 's':
                color = 'yellow' if event[2] > 0 else 'purple'
                plt.fill_betweenx(range(*shaded_y_lims), 
                                  event[1], events_list[idx-1][1], color=color, alpha=0.1)

        tot_return = round(100*(amount / init_amount - 1), 2)
        tot_return = str(tot_return) + '%'
        plt.title("%s Price Data\nThresh=%s\nTotal Amt: $%s\nTotal Return: %s"%(tickerSymbol, thresh, round(amount,2), tot_return), fontsize=20)
        plt.ylim(*y_lims)
        plt.show()
    

    return predictions, events_list

In [ ]:
my_order = (28,0,28)
my_thresh = 0.005
preds, events = run_simulation(returns, prices, amount=100, order=my_order, thresh=my_thresh)

In [ ]:
print(events)

In [ ]:
for date, pred_r in preds.items():
    acc_r = returns[date]
    print(date, "|", acc_r, "  ", pred_r)

In [ ]:
start_date = pd.to_datetime('2021-01-01 00:00:00-05:00')
end_date = pd.to_datetime('2021-06-01 00:00:00-05:00')

plt.figure(figsize=(10,4))
plt.xlim(start_date, end_date)
plt.plot(returns)
plt.plot(preds)
plt.ylabel('Return', fontsize=20)